#1(a)

In [1]:
import imblearn
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import hamming_loss
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline


In [21]:
from sklearn.model_selection import train_test_split
import pandas as pd

#df = pd.read_csv('Frogs_MFCCs.csv', header=0)
df = pd.read_csv('…/data/Frogs_MFCCs.csv', header=0)
X = df.iloc[:,:22]
y = df.iloc[:, 22:-1]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

#1b(i)

- Exact match means the prediction label is exactly matching with the true set of labels
- Hamming loss is the portion of the wrong label to the total number of labels

hamming loss is design for multi label classification 

#1b(ii)

In [ ]:
for col in ['Family', 'Genus', 'Species']:
  score = 0
  param_grid = {'C': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2],'gamma': [-3, -2, -1, 0, 1, 2,3,4,5,6]}  
  grid = GridSearchCV(SVC(), param_grid, refit = True, cv = 10, scoring = make_scorer(hamming_loss,greater_is_better=False)) 
  grid.fit(X, y[col])
  print(grid.best_params_, col)

Output:

{'C': 0.4, 'gamma': 6} Family

{'C': 1, 'gamma': 3} Genus

{'C': 1, 'gamma': 3} Species

#1b(iii)

In [ ]:
for col in ['Family', 'Genus', 'Species']:
  param_grid = {'LinearSVC__penalty':['l1'],'LinearSVC__C': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2]}  
  pipe = Pipeline([('scaler', StandardScaler()), ('LinearSVC', LinearSVC(penalty='l1', dual = False))])
  grid = GridSearchCV(pipe, param_grid, refit = True, cv = 10, scoring = make_scorer(hamming_loss,greater_is_better=False), ) 
  grid.fit(X, y[col])
  print(grid.best_params_, col)

Output:

{'LinearSVC__C': 0.1} Family

{'LinearSVC__C': 1.6} Genus

{'LinearSVC__C': 0.5} Species

#1b(iv)

In [ ]:

for col in ['Family', 'Genus', 'Species']:
  param_grid = {'LinearSVC__C': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2]}  
  pipe = Pipeline([('smote', SMOTE()),('scaler', StandardScaler()), ('LinearSVC', LinearSVC(penalty='l1', dual = False))])
  grid = GridSearchCV(pipe, param_grid, refit = True, cv = 10, scoring = make_scorer(hamming_loss,greater_is_better=False), ) 
  grid.fit(X, y[col])
  print(grid.best_params_, col)

{'LinearSVC__C': 1.7} Family

{'LinearSVC__C': 1.2} Genus

{'LinearSVC__C': 0.1} Species

#1b(v)

In [ ]:
#Classifier Chain

#1b(vi)

#2(a)(b)(c)

In [43]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score

#(a)
good_choice = -1
tmp = float('inf')
h_distance = []
h_loss = []
h_score = []

for k in range(2, 51):
  model = KMeans(n_clusters=k, random_state=0)
  t = silhouette_score(X, model.fit_predict(X))
  h_distance.append(t)
  if tmp > t:
    tmp = t
    good_choice = k
  print('when k is', k, ', the best k is', good_choice)
  
  #(b)
  model = KMeans(n_clusters=good_choice, random_state=0)
  labels = model.fit_predict(X)
  majority_family = [defaultdict(int) for _ in range(good_choice)]
  majority_genus = [defaultdict(int) for _ in range(good_choice)]
  majority_species = [defaultdict(int) for _ in range(good_choice)]
  for i in range(len(y)):
    majority_family[labels[i]][y.iloc[i][0]] += 1
    majority_genus[labels[i]][y.iloc[i][1]] += 1
    majority_species[labels[i]][y.iloc[i][2]] += 1

  majority_family = [max(i, key=lambda x:i[x]) for i in majority_family]
  majority_genus = [max(i, key=lambda x:i[x]) for i in majority_genus]
  majority_species = [max(i, key=lambda x:i[x]) for i in majority_species]

  print( 'majority family is',majority_family,'majority genus is',majority_genus, 'majority species is',majority_species)
  
  #c
  look_up = []
  for i,j,z in zip(majority_family, majority_genus,majority_species):
    look_up.append((i, j, z))

  predict_y = []
  for i in range(len(labels)):
    predict_y.append(look_up[labels[i]])
  
  
  loss = np.sum(np.not_equal(y.to_numpy(), predict_y))/float(y.size)
  score = np.sum(np.equal(y.to_numpy(), predict_y))/float(y.size)
  h_loss.append(loss)
  h_score.append(score)
  print('hamming loss is', loss,'hamming score is',score,'hamming distance is',t)

when k is 2 , the best k is 2
majority family is ['Leptodactylidae', 'Hylidae'] majority genus is ['Adenomera', 'Hypsiboas'] majority species is ['AdenomeraHylaedactylus', 'HypsiboasCordobae']
hamming loss is 0.2985406532314107 hamming score is 0.7014593467685893 hamming distance is 0.3486778410277152
when k is 3 , the best k is 2
majority family is ['Leptodactylidae', 'Hylidae'] majority genus is ['Adenomera', 'Hypsiboas'] majority species is ['AdenomeraHylaedactylus', 'HypsiboasCordobae']
hamming loss is 0.2985406532314107 hamming score is 0.7014593467685893 hamming distance is 0.36768245219926315
when k is 4 , the best k is 2
majority family is ['Leptodactylidae', 'Hylidae'] majority genus is ['Adenomera', 'Hypsiboas'] majority species is ['AdenomeraHylaedactylus', 'HypsiboasCordobae']
hamming loss is 0.2985406532314107 hamming score is 0.7014593467685893 hamming distance is 0.3788646212055977
when k is 5 , the best k is 2
majority family is ['Leptodactylidae', 'Hylidae'] majority g

In [44]:
import numpy as np

mean_hamming_distance = np.mean(h_distance)
std_hamming_distance = np.std(h_distance)
print('the mean of the 50 Hamming Distances is', mean_hamming_distance, ', standard deviation is', std_hamming_distance)

the mean of the 50 Hamming Distances is 0.26722928820839453 , standard deviation is 0.03344312058958613


#2(b)

In [45]:
mean_Hamming_score = np.mean(h_distance)
std_Hamming_score = np.std(h_distance)
print('the mean of the 50 Hamming score is', mean_Hamming_score, ', standard deviation is', std_Hamming_score)

the mean of the 50 Hamming score is 0.26722928820839453 , standard deviation is 0.03344312058958613


#2(c)

In [46]:
mean_Hamming_loss = np.mean(h_loss)
std_Hamming_loss = np.std(h_loss)
print('the mean of the 50 Hamming loss is', mean_Hamming_loss, ', standard deviation is', std_Hamming_loss)

the mean of the 50 Hamming loss is 0.13825171486245647 , standard deviation is 0.08068032662593413
